# Fully convolutional semantic segmentation

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

In this notebook, we retrain the VGG16 model to carry out 'fully convolutional' semantic segmentation

We will implement the approach of [Long et al. 2015](https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf)

![](figs/Slide47.PNG)

![](figs/Slide48.PNG)

We'll use the Grand Canyon sandbars data set. First we'll copy over the files from S3 to our local drive, then we'll train the model, and test the model outputs

![](figs/dl_tools_fullyconv.png)

## Preparing the data

### Looking at the S3 file structure

Let's load in our s3fs library and explore the contents of the 'fully_conv_semseg' folder

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
root = 'esipfed/cdi-workshop'

In [ ]:
fs.ls(root+'/fully_conv_semseg')

In [ ]:
fs.ls(root+'/fully_conv_semseg/data_gc/labels/gtFine/train/gc')

### Creating samples

You'll see a very specific folder structure for each of the datasets

* top_level (data_gc)
    * samples
        * RGB
            * train or val
                * site name (gc)
            
* top_level (data_gc)
    * labels
        * gtFine
            * train (or val)
                * site name (gc)            

In [ ]:
direc = root+'/fully_conv_semseg/data_gc/samples/RGB/train/gc'
files = [f for f in fs.ls(direc) if f.endswith('.png')]
len(files)

Create data structure for samples (images to train with)

In [ ]:
import os
os.mkdir('data') ##root+os.sep+
os.mkdir('data'+os.sep+'samples')  
os.mkdir('data'+os.sep+'samples'+os.sep+'RGB')
os.mkdir('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'train')
os.mkdir('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'val')
os.mkdir('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'train'+os.sep+'data')
os.mkdir('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'val'+os.sep+'data')

Next we need to copy over the files we need (this takes a while):

In [ ]:
from imageio import imread, imwrite
for file in files:
    print("working on ",file)
    with fs.open(file, 'rb') as fim:
        image = imread(fim)   
        imwrite('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'train'+os.sep+'data'+os.sep+file.split('/')[-1], image)

This next bit finds the 'validation' files and copies them over

In [ ]:
direc = root+'/fully_conv_semseg/data_gc/samples/RGB/val/gc'
files = [f for f in fs.ls(direc) if f.endswith('.png')]
len(files)

In [ ]:
for file in files:
    print("working on ",file)
    with fs.open(file, 'rb') as fim:
        image = imread(fim)   
        imwrite('data'+os.sep+'samples'+os.sep+'RGB'+os.sep+'val'+os.sep+'data'+os.sep+file.split('/')[-1], image)

### Creating labels

We recreate the file structure for the labels

In [ ]:
!pwd

In [ ]:
os.mkdir('data'+os.sep+'labels')
os.mkdir('data'+os.sep+'labels'+os.sep+'gtFine')
os.mkdir('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'train')
os.mkdir('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'val')
os.mkdir('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'train'+os.sep+'data')
os.mkdir('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'val'+os.sep+'data') 

In [ ]:
direc = root+'/fully_conv_semseg/data_gc/labels/gtFine/train/gc'
files = [f for f in fs.ls(direc) if f.endswith('.png')]
len(files)

In [ ]:
for file in files:
    print("working on ",file)
    with fs.open(file, 'rb') as fim:
        image = imread(fim)   
        imwrite('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'train'+os.sep+'data'+os.sep+file.split('/')[-1], image)

In [ ]:
direc = root+'/fully_conv_semseg/data_gc/labels/gtFine/val/gc'
files = [f for f in fs.ls(direc) if f.endswith('.png')]

for file in files:
    print("working on ",file)
    with fs.open(file, 'rb') as fim:
        image = imread(fim)   
        imwrite('data'+os.sep+'labels'+os.sep+'gtFine'+os.sep+'val'+os.sep+'data'+os.sep+file.split('/')[-1], image)

### Create a labeldefs.txt file

We need to tell the program what the classes are and what RGB color they correspond to in the label imagery

First, let's change directory and run the code in the 'semseg_fullyconv' folder

In [ ]:
import os
os.chdir('semseg_fullyconv/')

In [ ]:
!pwd

Specify labels and associated red, green, and blue colors

In [ ]:
labels = ['rock','water','veg','sand','other']
r = [102, 0, 0, 255, 255]
g = [51, 0, 255, 255, 0]
b = [0, 255, 0, 0, 0]

In [ ]:
with open('labeldefs.txt', 'a') as f:
    for item in range(len(labels)):
        f.write(labels[item]+','+str(r[item])+','+str(g[item])+','+str(b[item])+'\n')

In [ ]:
!cat labeldefs.txt

## Training the Net

For illustrative purposes, to save time we are going to train the model using just 15 epochs. For real applications, you would want to train for tens to hundreds of epochs

In [ ]:
out_direc = 'data_test10'
data_source = 'data'
data_dir = '../data'
num_epochs=15

While the model trains, we'll watch a video that explains what the VGG model is

In [ ]:
from IPython.display import YouTubeVideo
from datetime import timedelta

start=int(timedelta(hours=0, minutes=14, seconds=40).total_seconds())

YouTubeVideo("DAOcjicFr1Y", start=start)

Now we train the model by calling 'train.py'

It is actually set up to download the correct model (VGG 16) from the internet before retraining it

In [ ]:
%run train.py --name $out_direc --data-source $data_source \
                                 --data-dir $data_dir --epochs $num_epochs

### Tidying up

Remove the VGG graph

In [ ]:
!rm vgg.zip
!rm -rf vgg_graph/

## Testing the model

First we specify an output directory, and point the program to the location of the 'test' images

In [ ]:
samps_dir = '../data/samples/RGB/val/data'
output = 'test_output'

In [ ]:
%run ./infer.py --name $out_direc --samples-dir $samps_dir \
                                 --output-dir $output --data-source $data_source

Take a look in the output folder. The labeling looks 'blobby' because
* we didn't train the model for very long
* there is no post-processing

## Testing the model with CRF post-processing

This time we'll run the inference with CRF post processing to try to get more refined label images

In [ ]:
%run ./infer_crf.py --name $out_direc --samples-dir $samps_dir \
                                 --output-dir $output --data-source $data_source

Take a look in the output folder again - you'll see a new set of files

### Tidying up

To remove the model, tensorboard info, and outputs:

In [ ]:
!rm -rf data_test10
!rm -rf tb
!rm -rf test_output

Remove the labeldefs.txt file

In [ ]:
!rm labeldefs.txt

change directory

In [ ]:
os.chdir('..')
!pwd

Delete the 'data' directory that we copied over from S3 earlier

In [ ]:
!rm -rf data

## DL-tools

The equivalent function in DL-tools is called and is the same as used here. The key is getting the data in a strict format such as we did here. Then the functions are executed in the following sequence

1. ```python semseg_fullyconv\make_labels.py```
   * this function creates label rgb images from class data in the .mat files
2. ```python semseg_fullyconv\train.py```
    * this function trains the network
3. ```python semseg_fullyconv\infer.py``` or ```python semseg_fullyconv\infer_crf.py```
    * this function carries out the semantic segmentation